In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
# Sample data for launch sites
launch_sites_data = {
    "Launch Site": ["CCAFS LC-40", "VAFB SLC-4E", "KSC LC-39A", "CCAFS SLC-40"],
    "Lat": [28.5623, 34.6321, 28.5733, 28.5618],
    "Long": [-80.5774, -120.6108, -80.6469, -80.5772],
}

# Create a DataFrame
launch_sites_df = pd.DataFrame(launch_sites_data)

# Initialize a folium map centered on the average latitude and longitude
map_center = [launch_sites_df["Lat"].mean(), launch_sites_df["Long"].mean()]
launch_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each launch site
for _, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        popup=row["Launch Site"],
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(launch_map)

# Save the map to an HTML file
launch_map.save("launch_sites_map.html")

# Display the map file location
print("Map saved as launch_sites_map.html")

Map saved as launch_sites_map.html


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [12]:
# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Initialize the map
nasa_coordinate = [29.559684888503615, -95.0830971930759]  # Example: NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers for each launch site
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# Display the map
site_map

In [13]:
# Data for launch sites (Replace with actual data if available)
launch_sites_data = {
    "Launch Site": ["CCAFS LC-40", "VAFB SLC-4E", "KSC LC-39A", "CCAFS SLC-40"],
    "Lat": [28.5623, 34.6321, 28.5733, 28.5618],
    "Long": [-80.5774, -120.6108, -80.6469, -80.5772],
}

# Create a DataFrame
launch_sites_df = pd.DataFrame(launch_sites_data)

# Calculate distance from the equator (latitude 0°)
launch_sites_df["Distance from Equator (°)"] = launch_sites_df["Lat"].abs()

# Check proximity to equator (arbitrarily considering <= 10° as "near the equator")
launch_sites_df["Near Equator"] = launch_sites_df["Distance from Equator (°)"] <= 10

# Display the result
print(launch_sites_df[["Launch Site", "Lat", "Distance from Equator (°)", "Near Equator"]])

    Launch Site      Lat  Distance from Equator (°)  Near Equator
0   CCAFS LC-40  28.5623                    28.5623         False
1   VAFB SLC-4E  34.6321                    34.6321         False
2    KSC LC-39A  28.5733                    28.5733         False
3  CCAFS SLC-40  28.5618                    28.5618         False


In [17]:
# Add a manual column for proximity to the coast
# True is assigned based on knowledge that all these sites are near the coast
launch_sites_df["Near Coast"] = [True, True, True, True]  # Adjust if you have precise data

# Display the result
print(launch_sites_df[["Launch Site", "Lat", "Long", "Near Coast"]])

    Launch Site      Lat      Long  Near Coast
0   CCAFS LC-40  28.5623  -80.5774        True
1   VAFB SLC-4E  34.6321 -120.6108        True
2    KSC LC-39A  28.5733  -80.6469        True
3  CCAFS SLC-40  28.5618  -80.5772        True


In [18]:
# Combine proximity checks for equator and coast
summary = launch_sites_df[["Launch Site", "Near Equator", "Near Coast"]]

# Display the summary
print("Proximity Summary:")
print(summary)

Proximity Summary:
    Launch Site  Near Equator  Near Coast
0   CCAFS LC-40         False        True
1   VAFB SLC-4E         False        True
2    KSC LC-39A         False        True
3  CCAFS SLC-40         False        True


In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [16]:
marker_cluster = MarkerCluster()

In [19]:
# Assuming spacex_df is already defined and contains a 'class' column

# Define a function to determine marker color based on the class value
def assign_marker_color(launch_class):
    return "green" if launch_class == 1 else "red"

# Apply the function to create the 'marker_color' column
spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)

# Display the updated DataFrame to verify the new column
print(spacex_df[["class", "marker_color"]])

    class marker_color
0       0          red
1       0          red
2       0          red
3       0          red
4       0          red
5       0          red
6       0          red
7       0          red
8       0          red
9       0          red
10      0          red
11      0          red
12      0          red
13      0          red
14      0          red
15      0          red
16      0          red
17      1        green
18      1        green
19      0          red
20      1        green
21      1        green
22      1        green
23      0          red
24      1        green
25      1        green
26      0          red
27      0          red
28      1        green
29      1        green
30      1        green
31      1        green
32      0          red
33      0          red
34      0          red
35      0          red
36      1        green
37      0          red
38      1        green
39      1        green
40      0          red
41      1        green
42      1  

In [20]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [21]:
# Assuming spacex_df is defined and contains 'Launch Site', 'Lat', 'Long', 'class', and 'marker_color'

# Initialize the map centered on the average latitude and longitude of the launch sites
map_center = [spacex_df["Lat"].mean(), spacex_df["Long"].mean()]
updated_map = folium.Map(location=map_center, zoom_start=5)

# Add markers with colors based on the 'marker_color' column
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {row['class']}",
        icon=folium.Icon(color=row["marker_color"], icon="info-sign"),
    ).add_to(updated_map)

# Save the updated map as an HTML file
updated_map_path = "updated_launch_sites_map.html"
updated_map.save(updated_map_path)

# Display completion message
print(f"Updated map saved as {updated_map_path}")

Updated map saved as updated_launch_sites_map.html


In [23]:
from math import radians, sin, cos, sqrt, atan2
import pandas as pd
import folium

In [24]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)*2 + cos(lat1) * cos(lat2) * sin(dlon / 2)*2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Radius of Earth in kilometers
    R = 6371.0
    distance = R * c
    return distance

In [26]:
# Example proximities DataFrame
proximities_df = pd.DataFrame({
    'Proximity': ['Proximity A', 'Proximity B', 'Proximity C'],
    'Lat': [28.5734, 28.5012, 28.5833],
    'Long': [-80.6469, -80.5313, -80.6811]
})

In [29]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [30]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [34]:
import folium
from folium import DivIcon
from math import radians, sin, cos, sqrt, atan2

# Distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)*2 + cos(lat1) * cos(lat2) * sin(dlon / 2)*2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Radius of Earth in kilometers
    R = 6371.0
    return R * c

In [35]:
# Example launch site coordinates (replace with actual values)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Example closest coastline point coordinates (use MousePosition tool to find actual values)
coastline_lat = 28.56367
coastline_lon = -80.57163

In [36]:
# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")

Distance from launch site to coastline: 128.43 km


In [37]:
# Create a folium map centered at the launch site
map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(map)

# Add a marker for the closest coastline point
folium.Marker(
    [coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(map)

# Add a distance marker showing the calculated distance
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
).add_to(map)

# Display the map
map

In [39]:
# Example coordinates (replace with actual values as needed)
launch_site_coordinates = [28.5623, -80.5774]  # Launch site
coastline_coordinates = [28.56367, -80.57163]  # Closest coastline

# Combine the coordinates into a list for PolyLine
coordinates = [launch_site_coordinates, coastline_coordinates]

In [40]:
# Initialize the folium map
site_map = folium.Map(location=launch_site_coordinates, zoom_start=15)

# Add the PolyLine to the map
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(lines)

# Add markers for visualization
folium.Marker(
    location=launch_site_coordinates,
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

folium.Marker(
    location=coastline_coordinates,
    popup='Closest Coastline',
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# Display the map
site_map

In [47]:
%pip install folium geopy

In [48]:
import folium
from geopy.distance import geodesic

# Define the coordinates of the launch site
launch_site_coords = (28.5721, 77.2036)  # Example: Replace with actual coordinates of the launch site

# Define the coordinates of nearby locations
closest_city_coords = (28.6139, 77.2090)  # Example: Delhi
closest_railway_coords = (28.6467, 77.2113)  # Example railway station
closest_highway_coords = (28.5274, 77.1384)  # Example highway intersection

# Calculate distances to the launch site
city_distance = geodesic(launch_site_coords, closest_city_coords).kilometers
railway_distance = geodesic(launch_site_coords, closest_railway_coords).kilometers
highway_distance = geodesic(launch_site_coords, closest_highway_coords).kilometers

# Create a folium map centered on the launch site
map_object = folium.Map(location=launch_site_coords, zoom_start=12)

# Add markers for the launch site and the closest locations
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site",
    icon=folium.Icon(color="green", icon="rocket")
).add_to(map_object)

folium.Marker(
    location=closest_city_coords,
    popup=f"Closest City (Distance: {city_distance:.2f} km)",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(map_object)

folium.Marker(
    location=closest_railway_coords,
    popup=f"Closest Railway (Distance: {railway_distance:.2f} km)",
    icon=folium.Icon(color="purple", icon="train")
).add_to(map_object)

folium.Marker(
    location=closest_highway_coords,
    popup=f"Closest Highway (Distance: {highway_distance:.2f} km)",
    icon=folium.Icon(color="orange", icon="road")
).add_to(map_object)

# Draw lines between the launch site and the closest locations
folium.PolyLine([launch_site_coords, closest_city_coords], color="blue", weight=2.5).add_to(map_object)
folium.PolyLine([launch_site_coords, closest_railway_coords], color="purple", weight=2.5).add_to(map_object)
folium.PolyLine([launch_site_coords, closest_highway_coords], color="orange", weight=2.5).add_to(map_object)

# Display the map
map_object